In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [5]:
from expiries import dict_expiries

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [7]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{variation}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [8]:
bnf_1min = pd.read_csv("../data/nifty_min (2).csv")
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
# bnf_1min.tail()
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
]

In [9]:
bnf_1min.tail()

,index,datetime,o,h,l,c,v
777613,nifty,2025-05-30 15:25:00,24741.4000,24742.7000,24740.5000,24741.7000,0
777614,nifty,2025-05-30 15:26:00,24742.2500,24746.2000,24740.3000,24740.3000,0
777615,nifty,2025-05-30 15:27:00,24741.0500,24749.0500,24739.5000,24747.1500,0
777616,nifty,2025-05-30 15:28:00,24746.5500,24746.8000,24731.1000,24745.2500,0
777617,nifty,2025-05-30 15:29:00,24743.6000,24749.3000,24731.8500,24736.6500,0


In [10]:
days_traded = bnf_1min['datetime'].dt.date.unique().tolist()

In [11]:
from datetime import date

def get_monthly_expiry(input_date: date, index_symbol: str) -> date | None:
    """
    Returns the farthest expiry in the same month and year as input_date
    for the given index_symbol using dict_expiries.
    """
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None

    # Filter expiries for the same month and year as input_date
    same_month_expiries = [
        dt.date() for dt in expiries
        if dt.year == input_date.year and dt.month == input_date.month
    ]

    return max(same_month_expiries) if same_month_expiries else None


In [12]:
index_ = 'nifty'
tf_ = '1m'
offset_ = '0m'

if index_ == 'nifty':
    LOT_SIZE_ = 75
    STRIKE_SPREAD_ = 50
    INDEX_LEVERAGE_ = 8
    PORTFOLIO_ = 1_00_00_000

In [35]:
async def send_trade(strike, opt_type, expiry, dte, current_date, current_time, entry_price, qty, tradebook):
    trade = {
        'entry date': current_date,
        'entry time': current_time,
        'index': index_,
        'strike': strike,
        'expiry': expiry,
        'dte': dte,
        'option type': opt_type,
        'entry price': entry_price,
        'qty': qty,
    }
    tradebook.append(trade)
    return tradebook

async def take_exit(strike, opt_type, exit_date, exit_time, tradebook, exit_price):
    for trade in tradebook:
        if (
            trade['option type'] == opt_type and
            trade['strike'] == strike and
            'exit time' not in trade
        ):
            trade['exit date'] = exit_date
            trade['exit time'] = exit_time
            trade['exit price'] = exit_price
            
    return tradebook

In [60]:
async def execute(df, tf, offset):
    df['datetime'] = pd.to_datetime(df['datetime'])
    start_date = dt.date(2019, 1, 1)
    end_date = dt.date(2025, 5, 31)

    tradebook = []
    search_time = dt.time(10, 0)

    current_date = start_date

    take_new_trade = False
    expiry_found = False
    rebalance = False
    current_timestamp = dt.datetime.combine(current_date, search_time)
    position_active = False
    first_trade_of_month = False

    while current_date <= end_date:
        print(current_date)
        if current_date in days_traded:
            spot_df = df[(df['datetime'] >= current_timestamp) & (df['datetime'].dt.date == current_date)]
            spot_df = spot_df.reset_index()
            if not len(spot_df) > 1:
                current_date += dt.timedelta(days=1)
                continue
            if not expiry_found:
                spot_atm = spot_df['o'].iloc[0]
                expiry = get_monthly_expiry(current_date, index_)
                dte = (expiry - current_date).days
                if dte >= 0 and len(spot_df) > 1:
                    # print(len(spot_df))
                    # print(spot_df.to_string())
                    pct_away = max(1, (dte / 10))
                    ce_strike = int(round(spot_atm * ((100 + pct_away) / 100) / STRIKE_SPREAD_) * STRIKE_SPREAD_)
                    pe_strike = int(round(spot_atm * ((100 - pct_away) / 100) / STRIKE_SPREAD_) * STRIKE_SPREAD_)
                    expiry_found = True
                    print(f'Nearest Expiry : {expiry}, Spot ATM : {spot_atm}, DTE : {dte}')
                    print(f'CE Strike : {ce_strike}, PE Strike : {pe_strike}')
                    rebalance = False
                else:
                    current_date += dt.timedelta(days=1)
                    continue
        
            if current_date.day in [1, 2, 3] and not position_active and not first_trade_of_month:
                search_time = dt.time(10, 0)
                take_new_trade = True
                current_timestamp = dt.datetime.combine(current_date, search_time)
                first_trade_of_month = True
    
            if take_new_trade and dte >= 0:
    
                ce_df = await fetch_data(
                    index=index_,
                    expiry=expiry,
                    strike=ce_strike,
                    asset_class='C',
                    start_date=current_date,
                    start_time=search_time,
                    end_date=current_date,
                    end_time=dt.time(15, 30),
                )
                if ce_df is not None and not isinstance(ce_df, str):
                    ce_df = resample(ce_df, tf, offset)
                    ce_df_pandas = ce_df.to_pandas()
                    # print(ce_df_pandas.iloc[0])
                    entry_price_ce = ce_df_pandas.iloc[0]['o']
                    qty_ce = int(round((PORTFOLIO_ * INDEX_LEVERAGE_ / ce_strike) / LOT_SIZE_) * LOT_SIZE_)
                else:
                    entry_price_ce = float('nan')
                    qty_ce = 0
    
                pe_df = await fetch_data(
                    index=index_,
                    expiry=expiry,
                    strike=pe_strike,
                    asset_class='P',
                    start_date=current_date,
                    start_time=search_time,
                    end_date=current_date,
                    end_time=dt.time(15, 30),
                )
                if pe_df is not None and not isinstance(pe_df, str):
                    pe_df = resample(pe_df, tf, offset)
                    pe_df_pandas = pe_df.to_pandas()
                    # print(ce_df_pandas.iloc[0])
                    entry_price_pe = pe_df_pandas.iloc[0]['o']
                    qty_pe = int(round((PORTFOLIO_ * INDEX_LEVERAGE_ / pe_strike) / LOT_SIZE_) * LOT_SIZE_)
                else:
                    entry_price_pe = float('nan')
                    qty_pe = 0
    
                print(f'CE Entry Price: {entry_price_ce}, PE Entry Price: {entry_price_pe}')
    
                tradebook = await send_trade(
                    strike=ce_strike, 
                    opt_type='C', 
                    expiry=expiry,
                    dte=dte,
                    current_date=current_date, 
                    current_time=search_time, 
                    entry_price=entry_price_ce, 
                    qty=qty_ce, 
                    tradebook=tradebook,
                )
                tradebook = await send_trade(
                    strike=pe_strike, 
                    opt_type='P', 
                    expiry=expiry,
                    dte=dte,
                    current_date=current_date, 
                    current_time=search_time, 
                    entry_price=entry_price_pe, 
                    qty=qty_pe, 
                    tradebook=tradebook,
                )
    
                take_new_trade = False
                rebalance = False
                position_active = True
    
            if not take_new_trade:
                for i, row in spot_df.iterrows():
                    # print(spot_df.iloc[i])
                    if row['c'] >= ce_strike or row['c'] <= pe_strike:
                        # SL Hit, Rebalance !
                        print(f'Rebalancing @ {row["datetime"]}')
                        current_timestamp = row['datetime']
                        search_time = (row['datetime']).time()
                        take_new_trade = True
                        rebalance = True
                        expiry_found = False
                        
                        ce_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=ce_strike,
                            asset_class='C',
                            start_date=current_date,
                            start_time=search_time,
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        if ce_df is not None and not isinstance(ce_df, str):
                            ce_df = resample(ce_df, tf, offset)
                            ce_df_pandas = ce_df.to_pandas()
                            exit_price_ce = ce_df_pandas.iloc[0]['c']
    
                        pe_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=pe_strike,
                            asset_class='P',
                            start_date=current_date,
                            start_time=search_time,
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        if pe_df is not None and not isinstance(pe_df, str):
                            pe_df = resample(pe_df, tf, offset)
                            pe_df_pandas = pe_df.to_pandas()
                            exit_price_pe = pe_df_pandas.iloc[0]['c']
    
                        tradebook = await take_exit(
                            strike=ce_strike, 
                            opt_type='C', 
                            exit_date=current_date, 
                            exit_time=search_time, 
                            tradebook=tradebook, 
                            exit_price=exit_price_ce
                        )
    
                        tradebook = await take_exit(
                            strike=pe_strike, 
                            opt_type='P', 
                            exit_date=current_date, 
                            exit_time=search_time, 
                            tradebook=tradebook, 
                            exit_price=exit_price_pe
                        )
                        print(f'CE Exit {exit_price_ce} , PE Exit {exit_price_pe}')
                        position_active = False
                        current_timestamp = row['datetime']
                        search_time = (row['datetime']).time()
                        break
                        
                    elif (row['datetime'].date() == expiry) and (row['datetime'].time() == dt.time(15, 20)):
                        # Expiry Exit, No Rebalancing
    
                        ce_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=ce_strike,
                            asset_class='C',
                            start_date=current_date,
                            start_time=search_time,
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        if ce_df is not None and not isinstance(ce_df, str):
                            ce_df = resample(ce_df, tf, offset)
                            ce_df_pandas = ce_df.to_pandas()
                            exit_price_ce = ce_df_pandas.iloc[-1]['c']
    
                        pe_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=pe_strike,
                            asset_class='P',
                            start_date=current_date,
                            start_time=search_time,
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        if pe_df is not None and not isinstance(pe_df, str):
                            pe_df = resample(pe_df, tf, offset)
                            pe_df_pandas = pe_df.to_pandas()
                            exit_price_pe = pe_df_pandas.iloc[-1]['c']
    
                        tradebook = await take_exit(
                            strike=ce_strike, 
                            opt_type='C', 
                            exit_date=current_date, 
                            exit_time=dt.time(15, 20), 
                            tradebook=tradebook, 
                            exit_price=exit_price_ce
                        )
    
                        tradebook = await take_exit(
                            strike=pe_strike, 
                            opt_type='P', 
                            exit_date=current_date, 
                            exit_time=dt.time(15, 20), 
                            tradebook=tradebook, 
                            exit_price=exit_price_pe
                        )
                        expiry_found = False
                        rebalance = False
                        take_new_trade = False
                        position_active = False
                        first_trade_of_month = False
    
                if not rebalance:
                    current_date += dt.timedelta(days=1)
                    search_time = dt.time(10, 0)
                if rebalance:
                    current_timestamp += dt.timedelta(minutes=1)
                    search_time = current_timestamp.time()

        else:
            current_date += dt.timedelta(days=1)
            continue

    tb =  pd.DataFrame(tradebook)
    if len(tb)> 0:
        tb['points'] = tb['entry price'] - tb['exit price']
        tb['slippage'] = (tb['entry price'] + tb['exit price']) * 0.01
        tb['final points'] = tb['points'] - tb['slippage']
        tb['PnL'] = tb['qty'] * tb['final points']
        tb['portfolio'] = PORTFOLIO_
        tb['index leverage'] = INDEX_LEVERAGE_
        tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
    return tb

In [61]:
tb = await execute(bnf_1min, tf_, offset_)

2019-01-01
Nearest Expiry : 2019-01-31, Spot ATM : 10822.45, DTE : 30
CE Strike : 11150, PE Strike : 10500
CE Entry Price: 60.0, PE Entry Price: 79.5
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-12
2019-01-13
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-26
2019-01-27
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
Nearest Expiry : 2019-02-28, Spot ATM : 10851.35, DTE : 27
CE Strike : 11150, PE Strike : 10550
CE Entry Price: 70.7, PE Entry Price: 75.2
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
Nearest Expiry : 2019-03-28, Spot ATM : 10842.65, DTE : 27
CE 

In [62]:
tb.tail(500)

,entry date,entry time,index,strike,expiry,dte,option type,entry price,qty,exit date,exit time,exit price,points,slippage,final points,PnL,portfolio,index leverage,ROI%
864,2021-12-22,15:01:00,nifty,17100,2021-12-30,8,C,90.6000,4650,2021-12-23,09:53:00,155.8000,-65.2000,2.4640,-67.6640,-314637.6000,10000000,8,-3.1464
865,2021-12-22,15:01:00,nifty,16800,2021-12-30,8,P,92.6000,4725,2021-12-23,09:53:00,50.2500,42.3500,1.4285,40.9215,193354.0875,10000000,8,1.9335
866,2021-12-23,09:54:00,nifty,17250,2021-12-30,7,C,85.4000,4650,2021-12-24,10:04:00,36.8000,48.6000,1.2220,47.3780,220307.7000,10000000,8,2.2031
867,2021-12-23,09:54:00,nifty,16950,2021-12-30,7,P,82.3500,4725,2021-12-24,10:04:00,137.6000,-55.2500,2.1995,-57.4495,-271448.8875,10000000,8,-2.7145
868,2021-12-24,10:05:00,nifty,17100,2021-12-30,6,C,79.0500,4650,2021-12-27,14:52:00,123.0000,-43.9500,2.0205,-45.9705,-213762.8250,10000000,8,-2.1376
869,2021-12-24,10:05:00,nifty,16800,2021-12-30,6,P,85.2000,4725,2021-12-27,14:52:00,30.5000,54.7000,1.1570,53.5430,252990.6750,10000000,8,2.5299
870,2021-12-27,14:53:00,nifty,17300,2021-12-30,3,C,39.1500,4650,2021-12-30,15:20:00,0.1000,39.0500,0.3925,38.6575,179757.3750,10000000,8,1.7976
871,2021-12-27,14:53:00,nifty,16950,2021-12-30,3,P,56.2500,4725,2021-12-30,15:20:00,0.0500,56.2000,0.5630,55.6370,262884.8250,10000000,8,2.6288
872,2022-01-03,10:00:00,nifty,17800,2022-01-27,24,C,143.9000,4500,2022-01-04,15:16:00,283.0000,-139.1000,4.2690,-143.3690,-645160.5000,10000000,8,-6.4516
873,2022-01-03,10:00:00,nifty,16950,2022-01-27,24,P,100.4000,4725,2022-01-04,15:16:00,53.2000,47.2000,1.5360,45.6640,215762.4000,10000000,8,2.1576


In [63]:
tb['ROI%'].sum()

-117.69441787500001

In [64]:
tb.to_csv('condors.csv', index=False)

In [65]:
tb['DATETIME'] = tb['entry date'].astype(str) + ' ' + tb['entry time'].astype(str)
tb['DATETIME'] = pd.to_datetime(tb['DATETIME'])
tb['Trade Year'] = tb['DATETIME'].dt.year

In [66]:
tb['expiry'] = pd.to_datetime(tb['expiry'])
tb['entry date'] = pd.to_datetime(tb['entry date'])
tb['dte'] = (tb['expiry'] - tb['entry date']).dt.days

In [67]:
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,61.7210,128,62.5000,2.8096,-3.5444,-20.1440,3.0640,...
2020,-230.0456,544,53.1250,4.0688,-5.6013,-270.0655,-0.8518,...
2021,21.5235,200,58.0000,2.7763,-3.6650,-19.9293,1.0800,...
2022,-8.7273,240,56.2500,2.9749,-3.9080,-36.8925,-0.2366,...
2023,6.5023,80,61.2500,1.6219,-3.4749,-26.1481,0.2487,...
2024,8.9516,112,59.8214,2.4926,-3.6756,-38.3179,0.2336,...
2025,22.3801,60,60.0000,2.6149,-3.2617,-9.7475,2.2960,...
Overall,-117.6944,1364,56.5982,3.1929,-4.5309,-270.0655,-0.4358,...
